In [ ]:
# پیاده‌سازی MLP با PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_data_torch(path, device='cpu'):
    """
    Load CSV, split, standardize features, and return torch tensors.
    """
    df = pd.read_csv(path)
    feat_cols = ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms',
                 'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']
    X = df[feat_cols].values.astype('float32')
    y = df['House_Price'].values.astype('float32').reshape(-1, 1)

    # split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)

    # standardize X
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # to torch tensors
    X_train = torch.from_numpy(X_train).to(device)
    X_test = torch.from_numpy(X_test).to(device)
    y_train = torch.from_numpy(y_train).to(device)
    y_test = torch.from_numpy(y_test).to(device)
    return X_train, X_test, y_train, y_test

class SimpleMLP(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(SimpleMLP, self).__init__()
        # one hidden layer + tanh + output
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.net(x)

def run_pytorch_mlp(path, device='cpu'):
    # load
    X_train, X_test, y_train, y_test = load_data_torch(path, device)
    # model, loss, optimizer
    model = SimpleMLP(in_dim=X_train.shape[1], hidden_dim=10).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=1e-4)

    # training loop
    for epoch in range(500):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f"[Epoch {epoch+1}] train loss = {loss.item():.2f}")

    # evaluation
    model.eval()
    with torch.no_grad():
        preds = model(X_test)
        mse = criterion(preds, y_test).item()
    print(f"PyTorch MLP Test MSE: {mse:.2f}")

if __name__ == "__main__":
    run_pytorch_mlp('house_price_regression_dataset.csv', device='cpu')

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "C:\Users\Meysam\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\lib\c10.dll" or one of its dependencies.